In [ ]:
pip install -U nltk

In [ ]:
import numpy as np
import pandas as pd

import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument



In [ ]:
import nltk 
nltk.download('stopwords')

In [ ]:
!pip install pyldavis

In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [ ]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
stop_words = nltk.corpus.stopwords.words('english')

### pre hoc

In [ ]:
import pandas as pd
worksheet1 = gc.open('Final Dataset').get_worksheet(0) #generalized data worksheet
worksheet2 = gc.open('Final Dataset').get_worksheet(1) #filtered data worksheet
rows1 = worksheet1.get_all_values()
rows2 = worksheet2.get_all_values()


stackData1=pd.DataFrame.from_records(rows1,columns=rows1[0])
# stackData2=pd.DataFrame.from_records(rows2,columns=rows2[0])
stackData1.drop(0, inplace=True, axis=0)
# stackData2.drop(0, inplace=True, axis=0)
# stackData=[stackData1, stackData2]
# stackData=pd.concat(stackData,ignore_index=True)
stackData=stackData1
stackData['ProcessedBody']=stackData['ProcessedBody']+stackData['Tags']+stackData['Title']

feature=["ProcessedBody", "LOC", "QuestionLength",	"Url+ImageCount",	"Reputation",	"user_badge_bronze_counts",	"user_badge_gold_counts",	"user_badge_silver_counts",	"accept_rate"]
data=stackData[feature].values
y=stackData.Label.values

### post hoc

In [ ]:
import pandas as pd
worksheet1 = gc.open('Final Dataset').get_worksheet(0)
worksheet2 = gc.open('Final Dataset').get_worksheet(1)
rows1 = worksheet1.get_all_values()
rows2 = worksheet2.get_all_values()


stackData1=pd.DataFrame.from_records(rows1,columns=rows1[0])
# stackData2=pd.DataFrame.from_records(rows2,columns=rows2[0])


stackData1.drop(0, inplace=True, axis=0)
# stackData2.drop(0, inplace=True, axis=0)

stackData=stackData1
# stackData=[stackData1, stackData2]
# stackData=pd.concat(stackData,ignore_index=True)

stackData['ProcessedBody']=stackData['ProcessedBody']+stackData['Tags']+stackData['Title']

feature=["ProcessedBody", "LOC", "QuestionLength",	"Url+ImageCount",	"Reputation",	"user_badge_bronze_counts",	"user_badge_gold_counts",	"user_badge_silver_counts",	"accept_rate" , "view_count",	"answer_count",	"favorite_count",	"question_score",	"up_vote_count",	"First_answer_Interval","Accept_answer_Interval"]
#---------count------------#
stackData["view_count"] = pd.to_numeric(stackData["view_count"])
stackData["answer_count"] = pd.to_numeric(stackData["answer_count"])
stackData["favorite_count"] = pd.to_numeric(stackData["favorite_count"])
stackData["question_score"] = pd.to_numeric(stackData["question_score"])
stackData["up_vote_count"] = pd.to_numeric(stackData["up_vote_count"])
#---------Data------------,	"First_answer_date",	"Accepted_answer_date"#
stackData["creation_date"] = pd.to_datetime(stackData["creation_date"],unit='s')
stackData["First_answer_date"] = pd.to_datetime(stackData["First_answer_date"],unit='s')
stackData["Accepted_answer_date"] = pd.to_datetime(stackData["Accepted_answer_date"],unit='s')
#----------Date Interval ------#
stackData["First_answer_Interval"]=(stackData["First_answer_date"]-stackData["creation_date"])/pd.Timedelta(minutes=1)
stackData["Accept_answer_Interval"]=(stackData["Accepted_answer_date"]-stackData["creation_date"])/pd.Timedelta(minutes=1)

stackData["First_answer_Interval"] = stackData["First_answer_Interval"].apply(lambda x: -1 if x < 0 else x)
stackData["Accept_answer_Interval"]=stackData["Accept_answer_Interval"].apply(lambda x: -1 if x < 0 else x)

data=stackData[feature].values
y=stackData.Label.values


## Doc2vec Model

In [ ]:
def process_words(texts, stop_words=stop_words, allowed_tags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    bigram = gensim.models.Phrases(texts, min_count=20, threshold=100)
    trigram = gensim.models.Phrases(bigram[texts], threshold=100)
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram_mod = gensim.models.phrases.Phraser(trigram)
    texts = [[word for word in simple_preprocess(str(doc), deacc=True, min_len=3) if word not in stop_words] for doc in texts]
    texts = [bigram_mod[doc] for doc in texts]
    texts = [trigram_mod[bigram_mod[doc]] for doc in texts]
    
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_tags])
    
    texts_out = [[word for word in simple_preprocess(str(doc), deacc=True, min_len=3) if word not in stop_words] for doc in texts_out]    
    return texts_out

In [ ]:
def tagged_document(data):
  for i, d in enumerate(data):
    yield gensim.models.doc2vec.TaggedDocument(d,[i])


In [ ]:
def doc2vec(data,size):
    global vectorModel
    vectorModel=gensim.models.doc2vec.Doc2Vec(vector_size=size,min_count=2,epochs=30)
    vectorModel.build_vocab(data)
    vectorModel.train(data,total_examples=vectorModel.corpus_count,epochs=vectorModel.epochs)

In [ ]:
def testModel(data):
    vectorList=[]
    for d in data:
        vectorList.append(vectorModel.infer_vector(d))
    df=pd.DataFrame(vectorList)
    return df

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

def permanceMetrics(Y, pred, pred_prob):
    acc = accuracy_score(Y,pred)
    pre = precision_score(Y,pred,average='weighted')
    re = recall_score(Y,pred, average='weighted')
    f1 = f1_score(Y,pred, average='weighted')
    acrc = roc_auc_score(Y,pred_prob,multi_class= 'ovr')
    return acc, pre, re, f1, acrc

def avgMetric(met):
    res = np.array(met)
    acc = res[::5].mean()
    pre = res[1::5].mean()
    re = res[2::5].mean()
    f1 = res[3::5].mean()
    acrc = res[4::5].mean()
    return np.array([acc, pre, re, f1, acrc])

## Doc2Vec on Body

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=UserWarning) 
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.metrics import confusion_matrix

skf = StratifiedKFold(n_splits=10)

i=0
rm = []
for size in range(20,40):
    i=0
    print("for Size: ",size)
    print('Metrics :  Accuracy \t\tPrecision \t\tRecall \t\tF1-score \tAUC_ROC')
    for train_index, test_index in skf.split(data,y):
        i=i+1
        X_train, X_test = data[train_index], data[test_index]
        y_train, y_test = y[train_index], y[test_index]
        trainData=X_train.tolist()
        testData=X_test.tolist()
        processed_train=process_words(trainData)
        data_for_training=list(tagged_document(processed_train))
        doc2vec(data_for_training,size)
        processed_test=process_words(testData)
        train_vector=testModel(processed_train)
        test_vector=testModel(processed_test)
        print(len(X_train), len(X_test))
        print(train_index,test_index)

        rf = RF(n_estimators = 15, max_depth = 8, criterion='entropy', random_state = 42)
        rf.fit(train_vector,y_train)
        pred = rf.predict(test_vector)
        pred_prob = rf.predict_proba(test_vector)
        acc, pre, re, f1, arc = permanceMetrics(y_test, pred, pred_prob)
        print('Fold-',i,': ', acc, pre, re, f1,arc)
        rm += [acc, pre, re, f1,arc]
        cm = confusion_matrix(y_test, pred)
        print(cm)
    acc, pre, re, f1, acrc = avgMetric(rm)
    print('\nAverage: ', acc, pre, re, f1, acrc)


## Pre Hoc & Post Hoc

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=UserWarning) 
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.metrics import confusion_matrix
from sklearn.feature_selection import SelectFromModel

skf = StratifiedKFold(n_splits=10)

i=0
rm = []
for size in range(20,40):
    i=0
    print("for Size: ",size)
    print('Metrics :  Accuracy \t\tPrecision \t\tRecall \t\tF1-score \tAUC_ROC')
    for train_index, test_index in skf.split(data,y):
        i=i+1
        X_train, X_test = data[:,0][train_index], data[:,0][test_index]
        y_train, y_test = y[train_index], y[test_index]
        trainData=X_train.tolist()
        testData=X_test.tolist()
        processed_train=process_words(trainData)
        data_for_training=list(tagged_document(processed_train))
        doc2vec(data_for_training,size)
        processed_test=process_words(testData)
        train_vector=testModel(processed_train)
        test_vector=testModel(processed_test)
        X_train_others,X_test_others=data[:,1:][train_index], data[:,1:][test_index]

        X_train_others_df=pd.DataFrame([[x for x in col] for col in X_train_others],columns=feature[1:])
        X_test_others_df=pd.DataFrame([[x for x in col] for col in X_test_others],columns=feature[1:])

        X_all_train_df=pd.concat([X_train_others_df], axis=1)
        X_all_test_df=pd.concat([X_test_others_df], axis=1)

        X_train=X_all_train_df.values
        X_test=X_all_test_df.values


        rf = RF(n_estimators = 15, max_depth = 8, criterion='entropy', random_state = 42)
        rf.fit(X_train,y_train)
        feat_importances = pd.Series(rf.feature_importances_, index=X_all_train_df.columns)
        feat_importances.nlargest(20).plot(kind='barh')
        pred = rf.predict(X_test)
        pred_prob = rf.predict_proba(X_test)
        acc, pre, re, f1, arc = permanceMetrics(y_test, pred, pred_prob)
        print('Fold-',i,': ', acc, pre, re, f1,arc)
        rm += [acc, pre, re, f1,arc]
        cm = confusion_matrix(y_test, pred)
        print(cm)
    acc, pre, re, f1, acrc = avgMetric(rm)
    print('\nAverage: ', acc, pre, re, f1, acrc)
